# Enter Stock Name

In [ ]:
stock = input('Please enter Ticker Symbol(e.g. AAPL) & Press Enter: ')

Please enter Ticker Symbol(e.g. AAPL) & Press Enter: FORD


# Getting Data & Calculating Algorithm

In [ ]:
#import libraries
import pandas as pd
import time
import pandas as pd
import numpy as np
import requests 
import plotly.express as px
import plotly.graph_objects as go
 
# get data / handle time / convert to numpy array
now = int(time.time()) 
now =str(now)
days_delta = int(time.time()) - 12500000  #102 days of stock onfo
days =str(days_delta)
r = requests.get('https://finnhub.io/api/v1/stock/candle?symbol='+stock+'&from='+days+'&to='+now+'&resolution=D&token=c36j4jqad3ifoi8hsu50')
j = r.json() 
df = pd.DataFrame.from_dict(j)
df.t = (pd.to_datetime(df['t'],unit='s'))
df_close = df[['t','c']]
all_close = df_close['c'].values.astype(float)
 
# libraries
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
# model building
train=all_close[:85].reshape(-1, 1)
test=all_close[85:].reshape(-1, 1)
scaler=MinMaxScaler()
scaled_train=scaler.fit_transform(train)
scaled_test=scaler.transform(test)
 
n_input=84
n_features=1
 
train_generator=TimeseriesGenerator(scaled_train,
                                     scaled_train,
                                      n_input,
                                      batch_size=1)
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
 
custom_early_stopping = EarlyStopping(   
    monitor='loss', 
    patience=70, 
    min_delta=0.001, 
    mode='min' # was max
)
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(n_input,n_features),return_sequences=True))
model.add(LSTM(50,activation='relu',return_sequences=True))
model.add(LSTM(10,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')
 
##
 
model.fit(train_generator,epochs=140, callbacks=[custom_early_stopping])
 
test_predictions = []
#Select last n_input values from the train data
first_eval_batch = scaled_train[-n_input:]
#reshape the data into LSTM required (#batch,#timesteps,#features)
current_batch = first_eval_batch.reshape((1, n_input, n_features))
for i in range(len(test)):
# get prediction, grab the exact number using the [0]
  pred = model.predict(current_batch)[0]
# Add this prediction to the list
  test_predictions.append(pred)
# The most critical part, update the (#batch,#timesteps,#features
# using np.append(
# current_batch[:        ,1:   ,:] ---------> read this as
# current_batch[no_change,1:end,no_change]
# (Do note the second part has the timesteps)
# [[pred]] need the double brackets as current_batch is a 3D array
# axis=1, remember we need to add to the second part i.e. 1st axis
  current_batch = np.append(current_batch[:,1:,:],[[pred]],axis=1)
 
 
actual_predictions = scaler.inverse_transform(test_predictions)
rows =list(range(85, 100))
pred_df = pd.DataFrame(data=actual_predictions,index=rows)
 
future_train = all_close[15:].reshape(-1,1)
scaled_future_train=scaler.fit_transform(future_train)
n_input=84
n_features=1
 
future_generator=TimeseriesGenerator(scaled_future_train,
                                     scaled_future_train,
                                      n_input,
                                      batch_size=1)
custom_early_stopping = EarlyStopping(   
    monitor='loss', 
    patience=70, 
    min_delta=0.001, 
    mode='min'
)
 
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(n_input,n_features),return_sequences=True))
model.add(LSTM(50,activation='relu',return_sequences=True))
model.add(LSTM(10,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')
 
 
## custom stop was here
 
model.fit(future_generator,epochs=140, callbacks=[custom_early_stopping])
 
future_predictions = []
#Select last n_input values from the train data
first_eval_batch = scaled_future_train[-n_input:]
#reshape the data into LSTM required (#batch,#timesteps,#features)
current_batch = first_eval_batch.reshape((1, n_input, n_features))
for i in range(len(test)):
# get prediction, grab the exact number using the [0]
  pred = model.predict(current_batch)[0]
# Add this prediction to the list
  future_predictions.append(pred)
# The most critical part, update the (#batch,#timesteps,#features
# using np.append(
# current_batch[:        ,1:   ,:] ---------> read this as
# current_batch[no_change,1:end,no_change]
# (Do note the second part has the timesteps)
# [[pred]] need the double brackets as current_batch is a 3D array
# axis=1, remember we need to add to the second part i.e. 1st axis
  current_batch = np.append(current_batch[:,1:,:],[[pred]],axis=1)
 
 
future_actual_predictions = scaler.inverse_transform(future_predictions)
rows =list(range(100, 115))
future_pred_df = pd.DataFrame(data=future_actual_predictions,index=rows)

Epoch 1/140
1/1 [==============================] - 4s 4s/step - loss: 0.2666
Epoch 2/140
1/1 [==============================] - 0s 77ms/step - loss: 0.2383
Epoch 3/140
1/1 [==============================] - 0s 81ms/step - loss: 0.2215
Epoch 4/140
1/1 [==============================] - 0s 75ms/step - loss: 0.2024
Epoch 5/140
1/1 [==============================] - 0s 78ms/step - loss: 0.1814
Epoch 6/140
1/1 [==============================] - 0s 84ms/step - loss: 0.1588
Epoch 7/140
1/1 [==============================] - 0s 84ms/step - loss: 0.1348
Epoch 8/140
1/1 [==============================] - 0s 78ms/step - loss: 0.1088
Epoch 9/140
1/1 [==============================] - 0s 70ms/step - loss: 0.0807
Epoch 10/140
1/1 [==============================] - 0s 71ms/step - loss: 0.0514
Epoch 11/140
1/1 [==============================] - 0s 73ms/step - loss: 0.0227
Epoch 12/140
1/1 [==============================] - 0s 93ms/step - loss: 0.0018
Epoch 13/140
1/1 [==============================] -

# Visualize The Data

In [ ]:
fig = px.line(df,x=df.index, y='c',template='plotly_dark')
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df[0], mode="lines",name='Test_Prediction'))
fig.add_trace(go.Scatter(x=future_pred_df.index, y=future_pred_df[0], mode="lines",name='Future_Prediction'))
fig.update_layout(title=stock+' Stock Price LSTM prediction & performance for 15 test days')
fig.update_yaxes(title='$')
fig.update_xaxes(title='Days')
#fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()
#fig.to_html('pred.html')
fig.write_html(stock+' Pred.html')